In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv("Datasets/IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
def clean(data):
    return BeautifulSoup(data, 'html.parser').get_text()

In [4]:
df["cleaned"] = df["review"].apply(clean)

C:\Users\vcsre\AppData\Local\Temp\ipykernel_21272\239685728.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(data, 'html.parser').get_text()


In [5]:
df

,review,sentiment,cleaned
0,One of the other reviewers has mentioned that ...,positive,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,positive,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,I'm going to have to disagree with the previou...


In [6]:
df['cleaned_text'] = df['cleaned'].str.replace(r'\.{3,}', '', regex=True) 

In [7]:
df = df.drop(["review","cleaned"],axis=1)
df

,sentiment,cleaned_text
0,positive,One of the other reviewers has mentioned that ...
1,positive,A wonderful little production. The filming tec...
2,positive,I thought this was a wonderful way to spend ti...
3,negative,Basically there's a family where a little boy ...
4,positive,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...
49995,positive,I thought this movie did a down right good job...
49996,negative,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,negative,I am a Catholic taught in parochial elementary...
49998,negative,I'm going to have to disagree with the previou...


In [8]:
df["sentiment"] = df["sentiment"].replace({"positive": 1, "negative": 0})
df

C:\Users\vcsre\AppData\Local\Temp\ipykernel_21272\4171975237.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sentiment"] = df["sentiment"].replace({"positive": 1, "negative": 0})


,sentiment,cleaned_text
0,1,One of the other reviewers has mentioned that ...
1,1,A wonderful little production. The filming tec...
2,1,I thought this was a wonderful way to spend ti...
3,0,Basically there's a family where a little boy ...
4,1,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...
49995,1,I thought this movie did a down right good job...
49996,0,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,0,I am a Catholic taught in parochial elementary...
49998,0,I'm going to have to disagree with the previou...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer()
X = vector.fit_transform(df["cleaned_text"])
Y = df["sentiment"]

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(df["cleaned_text"],Y,test_size=0.2)
pipe = Pipeline([
    ("vector",CountVectorizer()),
    ("model",RandomForestClassifier())
])
pipe.fit(X_train,y_train)

Pipeline(steps=[('vector', CountVectorizer()),
                ('model', RandomForestClassifier())])

In [12]:
pipe.score(X_test,y_test)

0.8553

In [13]:
import pickle as pkl
with open("models/movie_reviews.pkl","wb") as file:
    pkl.dump(pipe,file)

In [14]:
import pickle as pkl
def predict(text):
    with open("models/movie_reviews.pkl", "rb") as file:
        model = pkl.load(file)
    output = ["nagative","positive"]
    return output[model.predict([text])[0]]

In [15]:
predict("this is good movie worth to watch")

'positive'

In [16]:
predict("this is bad movie not worth to watch")

'nagative'